
This is from the RDKit blog
http://rdkit.blogspot.com/2016/05/trying-out-py3dmol.html
<hr>

I first saw [3DMol.js](http://3dmol.csb.pitt.edu/) from Dave Koes' group at the ACS meeting in San Diego. It looked great, but I haven't had time to get back to it.

This morning Dave Tweeted about Jupyter integration for 3DMol.js and I just had to try it. This is the small experiment to see what's possible.

In [ ]:
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import Draw
from rdkit.Chem.Draw import IPythonConsole
from rdkit import rdBase
print(rdBase.rdkitVersion)
import os,time
print( time.asctime())

In [ ]:
import py3Dmol

Start with an example from the py3Dmol docs, just to make sure things are working:

In [ ]:
p = py3Dmol.view(query='pdb:1ycr')
p.setStyle({'cartoon': {'color':'spectrum'}})
p.show()

Demonstrate loading PDB data:

In [ ]:
#crambin = open('1CRN.pdb','r').read()
crambin = open('InitialAtomicStructure.pdb','r').read()
p = py3Dmol.view(width=400,height=400)
p.addModel(crambin,'pdb')
#p.setStyle({'cartoon': {'color':'spectrum'}})
p.setStyle({'stick':{}})
p.setBackgroundColor('0xeeeeee')
p.zoomTo()
p.show()

Here's a molecule from the RDKit distance geometry tests:

In [ ]:
m = Chem.AddHs(Chem.MolFromSmiles('COc1ccc2[C@H](O)[C@@H](COc2c1)N3CCC(O)(CC3)c4ccc(F)cc4'))
AllChem.EmbedMultipleConfs(m,useExpTorsionAnglePrefs=True,useBasicKnowledge=True)
mb = Chem.MolToMolBlock(m)
p = py3Dmol.view(width=400,height=400)
p.addModel(mb,'sdf')
p.setStyle({'stick':{}})
p.setBackgroundColor('0xeeeeee')
p.zoomTo()
p.show()

The thing I really want to be able to do is look at multiple molecules, so let's try showing all the conformers aligned to each other.

In [ ]:
core = m.GetSubstructMatch(Chem.MolFromSmiles('C1C(O)c2ccccc2OC1'))
print(core)
m

In [ ]:
AllChem.AlignMolConformers(m,atomIds=core)

In [ ]:
p = py3Dmol.view(width=400,height=400)
for conf in m.GetConformers():
    mb = Chem.MolToMolBlock(m,confId=conf.GetId())
    p.addModel(mb,'sdf')
p.setStyle({'stick':{}})
p.setBackgroundColor('0xeeeeee')
p.zoomTo()
p.show()

And lastly, let's look at results from the 3D alignment code:

In [ ]:
suppl = Chem.SDMolSupplier('ref_e2.sdf',removeHs=False)
ms = [x for x in suppl]
len(ms)

In [ ]:
Draw.MolsToGridImage([Chem.RemoveHs(x) for x in ms[:12]],molsPerRow=4)

Pick two of those molecules and generate new conformations for them, then display them to demonstrate that they really aren't aligned:

In [ ]:
refMol = ms[0]
prbMol = ms[10]
AllChem.EmbedMolecule(refMol,useExpTorsionAnglePrefs=True,useBasicKnowledge=True)
AllChem.EmbedMolecule(prbMol,useExpTorsionAnglePrefs=True,useBasicKnowledge=True)

p = py3Dmol.view(width=400,height=400)
p.addModel(Chem.MolToMolBlock(ms[0]),'sdf')
p.addModel(Chem.MolToMolBlock(ms[10]),'sdf')
p.setStyle({'stick':{}})
p.setBackgroundColor('0xeeeeee')
p.zoomTo()
p.show()

Now align and display them: